In [242]:
import torch
import torch.nn as nn
#from torch.nn.utils.rnn import pack_padded_sequence

class GatedRecurrentUnitHnNet(nn.Module):
    def __init__(self, input_size, output_size, 
                        window_size, hidden_size, num_layers, 
                        target_type_string='Regression',
                        bias=True, batch_first=True, 
                        bidirectional=False,
                        dropout_hidden=0, dropout_Hn=0):
        super().__init__()
        self.bidirectional = bidirectional
        self.seq_len = window_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            bias=bias,
                            batch_first=batch_first,
                            dropout=dropout_hidden, 
                            bidirectional=bidirectional)
        self.hn_dropout_layer = nn.Dropout(dropout_Hn)
        self.predict_layer = nn.Linear(hidden_size * num_layers * (bidirectional + 1), output_size) #(hidden*layers*1, 1)
        
        self.target_type_string = target_type_string
        if target_type_string=='Regression':
            self.loss_function = nn.MSELoss()
        elif target_type_string=='Classification':
            self.loss_function = nn.CrossEntropyLoss()
             
    def forward(self, input):
        print('batch:', input)
        print(input.size())
        #print(input[0])
        #print(input[0][0])

        gru_out, h_n = self.gru(input) #h_n shape: (num_layers * num_directions, batch, hidden_size)
        print('h_n:', h_n)
        print('h_n size:', h_n.size(0), h_n.size(1), h_n.size(2))
        h_n_transpose = torch.transpose(h_n, 0, 1).contiguous() #shape: (batch, num_layers * num_directions, hidden_size)
        print('h_n_transpose:', h_n_transpose)
        print('h_n_transpose size:', h_n_transpose.size(0), h_n_transpose.size(1), h_n_transpose.size(2))
        
        h_n_view = h_n_transpose.view(input.size(0), -1) #shape: (batch, num_layers * num_directions * hidden_size)
        print('h_n_view:', h_n_view)
        print('h_n_view size:', h_n_view.size(0), h_n_view.size(1))
        
        h_n_dropout = self.hn_dropout_layer(h_n_view)
        predict = self.predict_layer(h_n_dropout) #predict est vertical
        #print('predict:', predict)
        #print('predict size:', predict.size())
        return predict

In [243]:
input_size = 2
output_size = 1
window_size = 2
hidden_size = 8
num_layers = 3
target_type_string='Regression'
bias=True
batch_first=True
bidirectional=True
dropout_hidden=0
dropout_Hn=0

In [244]:
gru = GatedRecurrentUnitHnNet(input_size, output_size, 
                        window_size, hidden_size, num_layers, 
                        target_type_string,
                        bias, batch_first, 
                        bidirectional,
                        dropout_hidden, dropout_Hn)

In [245]:
inputs = [[[7.9018e-04, 1.2779e-01],[7.9018e-04, 1.2779e-01]],
        [[5.9300e-04, 1.0390e-01],[5.9300e-04, 1.0390e-01]]]

In [246]:
gru(torch.Tensor(inputs))

batch: tensor([[[0.0008, 0.1278],
         [0.0008, 0.1278]],

        [[0.0006, 0.1039],
         [0.0006, 0.1039]]])
torch.Size([2, 2, 2])
h_n: tensor([[[-0.1338,  0.0955, -0.0209, -0.0913, -0.0366, -0.1782, -0.2123,
          -0.1955],
         [-0.1291,  0.0997, -0.0187, -0.0849, -0.0343, -0.1804, -0.2111,
          -0.1968]],

        [[-0.0472,  0.1072,  0.1520,  0.0925,  0.0693, -0.2980, -0.1162,
          -0.0873],
         [-0.0481,  0.1090,  0.1542,  0.0952,  0.0638, -0.3011, -0.1202,
          -0.0824]],

        [[-0.2283,  0.1388, -0.1411, -0.0077, -0.2246,  0.0055,  0.1565,
           0.0815],
         [-0.2270,  0.1399, -0.1395, -0.0109, -0.2270,  0.0048,  0.1546,
           0.0838]],

        [[ 0.0263, -0.0926, -0.0592, -0.2959,  0.0132, -0.0678,  0.1275,
          -0.1417],
         [ 0.0268, -0.0952, -0.0589, -0.2967,  0.0140, -0.0641,  0.1273,
          -0.1432]],

        [[-0.0621, -0.2374,  0.1402, -0.0332,  0.1971,  0.0300,  0.1999,
           0.3470],
         

tensor([[-0.0327],
        [-0.0320]], grad_fn=<AddmmBackward>)

In [197]:
d1 = torch.Tensor([[1,2,3,4,5], [6,7,8,9,10]])
d2 = torch.Tensor([[11,12,13,14,15], [16,17,18,19,20]])
d1.size()

torch.Size([2, 5])

In [192]:
d1

tensor([[ 1.,  2.,  3.,  4.,  5.],
        [ 6.,  7.,  8.,  9., 10.]])

In [193]:
d2

tensor([[11., 12., 13., 14., 15.],
        [16., 17., 18., 19., 20.]])

In [205]:
cat = torch.cat((d1, d2), dim=1)

In [206]:
cat

tensor([[ 1.,  2.,  3.,  4.,  5., 11., 12., 13., 14., 15.],
        [ 6.,  7.,  8.,  9., 10., 16., 17., 18., 19., 20.]])

In [207]:
cat.size()

torch.Size([2, 10])

In [209]:
d1.view(-1)

tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])